In [10]:
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

In [11]:
# Load Packages
import tensorflow as tf
from tensorflow.keras import backend
#from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
from keras.initializers import glorot_uniform
from keras.models import load_model, save_model
import keras
import numpy as np
import pandas as pd
import random
import sys
import io
import re
import os

In [12]:
#os.chdir('/content/drive/My Drive/Colab Notebooks/NLP Group Project/')

In [13]:
# Read Songs
songs = pd.read_csv('drake-songs.csv')

In [14]:
text = ''

for index, row in songs['lyrics'].iteritems():
    cleaned = str(row).lower().replace(' ', '\n')
    text = text + " ".join(re.findall(r"[a-z']+", cleaned))
    
len(text)

367372

In [15]:
tokens = re.findall(r"[a-z'\s]", text)

chars = sorted(list(set(tokens)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 28


In [16]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('nb sequences:', len(sentences))

nb sequences: 122444


In [17]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [18]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
model = tf.keras.models.load_model('drake_character_model.h5')

In [20]:
def generate_text(model, seed, length, diversity):

    maxlen = 40
    generated = ''
    sentence = text[seed: seed + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
    
    print('----- Actual Text -----')
    print(text[seed:seed+maxlen+length])
    
    # JG - Added to return generated and actual text for comparison
    return generated, text[seed:seed+maxlen+length]

In [21]:
import warnings
warnings.filterwarnings('ignore')
generate_text(model, 123, 200, 0.5)

----- Generating with seed: "hoes i suppose i just wanna be i just wa"
hoes i suppose i just wanna be i just wanna fear a blowing you drangy in the niggas and ickin' in the presing dropin' i always sorry you should got me i be i got me i'm just was a better sen my face you just got it one a shot the bangs and 
----- Actual Text -----
hoes i suppose i just wanna be i just wanna be successful i just wanna be i just wanna be successful i just wanna be i just wanna be successful drizzy ah yeah trey i fuckin' feel you they be starin' at the money like it's unfamiliar i get i


("hoes i suppose i just wanna be i just wanna fear a blowing you drangy in the niggas and ickin' in the presing dropin' i always sorry you should got me i be i got me i'm just was a better sen my face you just got it one a shot the bangs and ",
 "hoes i suppose i just wanna be i just wanna be successful i just wanna be i just wanna be successful i just wanna be i just wanna be successful drizzy ah yeah trey i fuckin' feel you they be starin' at the money like it's unfamiliar i get i")

Predicted!

## Evaluation - Edit Distance

In [22]:
# Packages for Evaluation
from nltk.metrics import *

In [ ]:
# Dictionary to store results
eval_dict = {}

# Loop through different diversity parameters to find the best one
for diversity in [0.2, 0.4, 0.6, 0.8]:

    total_distance = 0
    lines_generated = 0

    # Loop through seed values starting at 1 and stepping 10000 (creates 37 rounds per diversity)
    for seed in range(1, len(text) - maxlen, 1000):

        # Get generated and actual text from model
        generated_text, actual_text = generate_text(model, seed, 200, diversity)

        # Use edit distance to see difference in characters generated
        total_distance += edit_distance(generated_text[maxlen:200+maxlen], actual_text[maxlen:200+maxlen])

        lines_generated += 1

        # Get average distance
        avg_dist = total_distance/lines_generated

    # Append results for each level of diversity to dictionary
    eval_dict[diversity] = avg_dist

----- Generating with seed: "oney money cars cars clothes clothes the"
oney money cars cars clothes clothes they should take the same that i would take the same they can't see it they should take you here to the shit i stay to take it that i would take me that i just got to take the do and i was torn i stand t
----- Actual Text -----
oney money cars cars clothes clothes the hoes i suppose yeah i want the money money and the cars cars and the clothes the hoes i suppose i just wanna be i just wanna be successful i just wanna be i just wanna be successful i just wanna be i
----- Generating with seed: "t i'm alive for it i want the money mone"
t i'm alive for it i want the money money to me i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it you know you
----- Actual Text -----
t i'm alive for it i want the money money and the cars cars and the clothes the hoes i 

i'm goin' in i'm goin' in and i'mma go harg i got a never the drown and i stay to take it is to the way it i don't can get it i get it i get it you do the way it i get it i get it i get it you don't do it i know you don't feel the cready ho
----- Actual Text -----
i'm goin' in i'm goin' in and i'mma go hard and i'mma go hard and i'mma go hard and i'mma go hard i'm goin' in i'm goin' in i'm goin' in i'm goin' in and i'mma go hard and i'mma go hard and i'mma go hard and i'mma go hard hold up wait one m
----- Generating with seed: " goin' in i'm goin' in i'm goin' in and "
 goin' in i'm goin' in i'm goin' in and i know they got me i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it i get it you know they got me i get it i get it i get it i get it i get i
----- Actual Text -----
 goin' in i'm goin' in i'm goin' in and i'mma go hard and i'mma go hard and i'mma go hard and i'mma go hard bitch i'm goin' in i'm goin' in i'm goin' in i'm g

t now that i didn't know last year who they want to you i stand the controking to the same that i wouldn't trust i want to take this who talk to you i got it i got to take it to you i got the same they should take you ang i want to take it 
----- Actual Text -----
t now that i didn't know last year who the fuck are y'all i swear it feels like the last few nights we've been everywhere and back but i just can't remember it all what am i doing what am i doing oh yeah that's right i'm doin' me i'm doin' 
----- Generating with seed: "they treat me like a legend am i really "
they treat me like a legend am i really money to the clay i got the swanter than i do to you i got the days i swear the pack in the centy i got the same that i comin' the presing in the caty i got the clang i stand that i got you to the try
----- Actual Text -----
they treat me like a legend am i really this cold i'm really too young to be feeling this old it's about time you admit it who you kiddin' and nobody's ever d

ake that fuckin dress off i swear you would be the beater i stand that i get it i do take you in the beat that i can't say it i be the way in the same they look that i stay to you i swear the crown to the same they want the do to take the s
----- Actual Text -----
ake that fuckin dress off i swear you won't forget me you'll be happy that you let me lay you down down down tryin to lay you down down down you still the baddest girl around round round they notice they notice let me lay you down down down
----- Generating with seed: "ross your mind because all this shit is "
ross your mind because all this shit is and you don't do it i told you i want to you i know that i don't need you to my flip it i do you was we got you i got a never things i know that you get to take the beats i got it i get it i stand the
----- Actual Text -----
ross your mind because all this shit is new to me i'm learning to behave but still spending every dollar they encourage me to save but i'm good i know that ni

 now uh go 'head thank me later yeah i know you know that i don't get it i want to you you know they got to you i got a worst you i got it i do the cribese i got the deam i got to take the do you do it they don't know how to the get the get
----- Actual Text -----
 now uh go 'head thank me later yeah i know what i said but later doesn't always come so instead it's okay you could thank me now uhh shallow nigga but deep enough to have gone swimming i think i'm spending all my time with the wrong women 
----- Generating with seed: "rried at hello because of what they allo"
rried at hello because of what they allound to the swerome though you do the go to take the beated to stay to the swerome on the days you got to take it i got a grammy i real i stick it the dozang i can't can tell the style they better on t
----- Actual Text -----
rried at hello because of what they allow where are they all now i know there somewhere watching so i'm trying to make them proud searching for the meaning yo

...

me for haiti what up zoe weezy f baby and that you want that ya i hope the bangs in your wavervy you would thank the cribed to my city you ang i'm a go the crown to told me and they just do your scenced you show you get me and i don't do yo
----- Actual Text -----
me for haiti what up zoe weezy f baby and the 'f' is for front door cause that's where i bring it soo woo if you bangin' muthafucka yeah girl it's on you know what it is when i finally make it home i just hope you miss me a little when i'm 
----- Generating with seed: " your lovin' i better find your heart i "
 your lovin' i better find your heart i just want to take it to took the boy you need to dry the girls we do you i want it you got to talk no confect in the alwhyp you take me i was to you i got you ang i can tell that shit that you want it
----- Actual Text -----
 your lovin' i better find your heart i bet if i give all my love then nothing's gonna tear us apart i'm more than just a number hey hey hey i doubt you'll fi

ng fake ids and the trash we going straight that's all the get it that you do bet i play in the say that you want to drank to the rushin' you think i swear your sway i should coming back you when i was you you play me to to the like your ca
----- Actual Text -----
ng fake ids and the trash we going straight to the top rooftop glows with a handful of girls and they all so foreign rain so poisoned rainbows flowing light skinned chick first flight from poland a whole lot of friends first flight from pol
----- Generating with seed: "ealest niggas say your lyrics do shit fo"
ealest niggas say your lyrics do shit for you you like you see in the picture it wasn't tripped the same stay in your way i don't drop it you want it i just want to you was back to the swange i'm armoury that you get money to get the swange
----- Actual Text -----
ealest niggas say your lyrics do shit for me i told my story then made history tell them imma need reservations for i've never really been one for the preserv

s fuck and you b b b bet i am all of the case i feel the times that's the same inters i want to marce thing controkitions and i got a gran money to my ingiars so in my like i scerting that i can't tell that you you to take the do you 'in't 
----- Actual Text -----
s fuck and you b b b bet i am all of them bitches i'm badder than mansions in malibu babylon but i never mention everything i dabble in and i always ride slow when i'm straddling and my shit's so wet you gotta paddle in gotta r r row gotta 
----- Generating with seed: "ze lord knows it's your worst nightmare "
ze lord knows it's your worst nightmare that i do the they can't see it i get it i get it i get it i get it i get it i get it you drop the pack and i do it always six in the best i out to take me i swear the beds i do you six in the benn th
----- Actual Text -----
ze lord knows it's your worst nightmare it's my first night here and this girl right here who knows what she knows so i'm going through her phone if she go to

 learned since getting richer i learned i and her and that you want to the how to worse the bears i ain't all i down if's no and i find your haard to the club they be to take you in my long as the dears it i do it in we just do you got the 
----- Actual Text -----
 learned since getting richer i learned working with the negatives could make for better pictures i learned hennessy and enemies is one hell of a mixture even though it's fucked up girl i'm still fucking wit' ya damn is it the fall time for
----- Generating with seed: "w bitch if you wave then i will surf you"
w bitch if you wave then i will surf your cry you want you i got the d tha money make me ride you ang you don't want you i can't see your jester to they cet on the same bands i don't take it i do it why i don't wanna got me to you i always 
----- Actual Text -----
w bitch if you wave then i will surf you i flew jet she flew commercial but we still met later that night after my session she came over i was aggressive and 

d you get an e for effort i piece letter than we got a does that they mozy a new to the do you do you better on play my way it and i don't mean i was stray niggas and that i get it that i spand this sworks to the class they trazy on the str
----- Actual Text -----
d you get an e for effort i piece letters together and get to talkin' reckless owl chains like credentials you know you see the necklace my life's a completed checklist i'm tired of hearin' 'bout who you checkin' for now just give it time w
----- Generating with seed: "ne except yourself and if you end up nee"
ne except yourself and if you end up neeted it how to the same with you with the pack than i stand a sockzy my ingiars so i stay through straight i know you i mean i was back they better smoking the bring to the girls with you you could go 
----- Actual Text -----
ne except yourself and if you end up needin' some extra help then i could help you know back on your feet and shit tryna get my karma up fuck the guilty and g

In [ ]:
# Look at results for different diversity levels
eval_dict

##  Evaluation - Artist Vocabulary

In [ ]:
# List of all generated words
generated_sentences = []

# Loop through seed values starting at 1 and stepping 10000 (creates 37 rounds per diversity)
for seed in range(1, len(text) - maxlen, 1000):
  
    # Get generated and actual text from model - specify diversity
    generated_text, actual_text = generate_text(model, seed, 200, 0.2)

    # Just add the characters generated by the model
    generated_sentences.append(generated_text[maxlen:maxlen+200])

generated_sentences

In [ ]:
# Create list to hold all generated words
generated_words = []

# Loop through generated sentences and make list of generated words
for generated_sentence in generated_sentences:
    generated_words += generated_sentence.split()

# Get unique generated words with set()
generated_words_unique = set(sorted(generated_words))
len(generated_words_unique)

In [ ]:
# Get all of artist's words from his/her lyrics for comparison
all_artist_words = text.replace("'", '').split(' ')

# Get unique artist words with set()
artist_words_unique = set(sorted(all_artist_words))
len(artist_words_unique)

In [ ]:
# Ratio of generated words that are words the artist actually uses
len(generated_words_unique.intersection(artist_words_unique)) / len(generated_words_unique)